# Distribution and Health Impacts of Brownfield Sites in Detroit, Michigan

At the turn of the 20th century, Detroit developed into one of the largest cities in the United States as an industrial hub for automobile production and other forms of manfuacturing Due to Detroit’s history as a manufacturing powerhouse alongside its eventual decline, many highly polluting or abandoned industrial sites called ‘brownfields’ are commonly found within city limits. Land surrounding brownfield sites are frequently contaminated with serum metals and can lead to long-term health impacts on the surrounding population. With the use of open data from the PHOENIX Project, we can observe various health demographics including life expectancy and asthma rates on a census-tract level. This study, with the use of Python and Geopandas, will aim to determine the number of brownfield sites within Detroit’s city limits, the amount of residential land within proximity of a brownfield site, and the health impacts of brownfield sites on surrounding communities. Notable patterns, including the distribution of 4 health conditions (asthma, cancer, COPD, and coronary heart disease) will be analyzed. Limitations found within the scope of the final project will also be discussed. 

## Background and Research Question
During the Industrial Revolution in the late 19th century, Detroit experienced population and industrial growth. Due to the city's location along the Detroit River, allowing direct access to the Great Lakes, it was seen as a valuable location for transporting manufactured goods via the Great Lakes or by rail to surrounding Great Lakes manufacutring hubs. At the turn of the 20th century, Detroit's influence expanded with the rise of the Ford Motor Company and the automobile industry, giving Detroit its well-known nickname "The Motor City". This period of prosperity however would change after the departure of the manfacutring industry to overseas markets, leading to a period of economic decline that has extended into present-day. In recent years, Detroit has returned to an upward trajectory, however elements of its industrial past are still present throughout city limits.

Since the departure of the manufacturing sector within Detroit and much of the Rust Belt region, formerly industrial sites were abandoned and now sit vacant. These types of abandoned sites, in addition to other heavily polluting industrial sites, are commonly referred to as "brownfields". The environmental and health impacts of brownfields haven't been commonly observed until in recent decades, where many studies including Evans K. Lodge's journal *The effect of residential proximity to brownfields, highways, and heavy
traffic on serum metal levels* have began to uncover the impacts brownfields and other pollutant sources can have on the surrounding residential population. 

This report aims to conduct a similar goal of observing the health impacts of brownfields. 3 main questions will be answered:
- 1. How many brownfield sites are present within the city of Detroit?
- 2. What percentage of residential area is within close proximity to a brownfield site?
- 3. How have brownfield sites impacted the long-term health of surrounding communities?

## Methodology

This section will discuss the datasets and analysis methods used to answer the 3 research questions. Snippets of Python code will be included within this section to show the functions used. The entire code can be found in the repository as *brownfields.py*.

### Datasets and Analysis Methods

Some of the datasets being used include sources from open data portals dedicated to the city of Detroit. A point-layer shapefile containing all the brownfield sites within Detroit city limits was utilized to visualize the locations of all brownfield sites. A shapefile containing the zoning codes used by the city and a shapefile of Detroit's boundaries were also acquired from their open data portals. TIGER shapefiles consisting of all census tracts within Michigan from 2019 were acquired from the US Census Bureau.

In addition to geographic datasets, health data was utilized to determine the trend of health demographics within Detroit. This health data was acquired from Wayne State University's PHOENIX Project, a project ran by the university's physicians and epidemiologists to acquire up-to-date health data within the state of Michigan. 5 health data statistics were acquired from their interactive map (life expectancy, asthma, COPD, coronary heart disease, and cancer). Each of the 5 tables were acquired in a census tract format.

2 methods were used within Python to conduct the analysis:
- Geopandas, a geospatial version of pandas that can read shapefiles, conduct spatial overlays/intersections, and add or remove data columns within a dataset.
- Areal allocation, a method found in dasymetric mapping, specific values from one dataset can be reallocated to fit the geometries of another dataset based on parameters including area and/or weighting.

### Data Processing and Analysis within Python

This section will follow the steps taken within Python to conduct the analysis. Some of the steps include:
- importing extensions and shapefiles
- filtering and cleaning data
- integrating health data to shapefiles
- calculating area of polygons
- converting string columns to integers
- conducting areal allocation to residential shapefile

The first step taken is to import the needed extensions and shapefiles into Python. In addition, shapefiles are converted into the study area's CRS, NAD 1983 State Plane Michigan South with the use of to_crs and the variable crs to store the CRS' EPSG ID.

In [ ]:
# Importing extensions
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# Defining study's CRS, Michigan State Plane South
crs = 2253

# Reading shapefiles and data tables
brownfields = gpd.read_file('Brownfield_Sites/brownfield_sites.shp').to_crs(crs)
tracts = gpd.read_file('tl_2019_26_tract/tl_2019_26_tract.shp').to_crs(crs)
city_boundary = gpd.read_file('City_of_Detroit_Boundary/City_of_Detroit_Boundary.shp').to_crs(crs)
zoning = gpd.read_file('Zoning_Official/zoning_official.shp').to_crs(crs)
asthma = gpd.read_file('Asthma.csv')
cancer = gpd.read_file('Cancer.csv')
copd = gpd.read_file('COPD.csv')
chd = gpd.read_file('Coronary_Heart_Disease.csv')
life_expect = gpd.read_file('Life_Expectancy.csv')

After all shapefiles and extensions are properly imported, we first scale the census tract shapefile down to the city of Detroit. This is done by filtering the census tract shapefile twice: first down to Wayne County and then down to Detroit city limits.

In [ ]:
# Filtering census tract shapefile to only include tracts within Wayne County
wayne_tracts = tracts[tracts['COUNTYFP'] == '163']

# Filtering again, this time to only include tracts within Detroit city limits
detroit_tracts = gpd.clip(wayne_tracts,city_boundary)

The next shapefile to filter is the brownfield data. Since this dataset is from 2014, the Detroit land use data will be utilized to determine which brownfield sites are no longer industrial. This is accomplished by selecting all points within the brownfield data that intersect with a non-industrial or polluting land use and then subtracting them from the dataset using gpd.overlay difference.

Now that past brownfield sites are removed, a 1500-foot buffer can be created around the existing brownfield sites. The buffers are all merged into one dataset and then converted into a GeoDataFrame due to the buffer function converting the Dataframe into a GeoSeries.

In [ ]:
# Filtering points shapefile by filtering zoning shapefile temporarily 
zoning_temp = zoning[zoning['ZONING_REV'].str.startswith('P') | zoning['ZONING_REV'].str.startswith('R') | zoning['ZONING_REV'].str.startswith('S')]

# Filter points shapefile with temporary zoning shapefile
current_brownfields = gpd.overlay(brownfields,zoning_temp,how='difference')

# Creating a 1500 ft buffer around existing brownfields
buffer = current_brownfields.geometry.buffer(1500)

# Merging all buffers into a multipolygon
buffer = buffer.union_all()

# Saving buffer as a geodataframe because buffer function converted it into a GeoSeries
buffer_gdf = gpd.GeoDataFrame(geometry=[buffer], crs=crs)

The next step of this analysis is to integrate the health data tables into the census tract shapefile. This is done by using .merge() to temporarily merge select columns from each respective table using a column from each shapefile that has matching values. In this case, the census tract and health data tables have matching census tract IDs, which are used to match the values to each respective census tract. A new column that contains the needed values is then created before removing the merged columns from the census tract data. In addition to life expectancy, the total population of each census tract was acquired from the life expectancy table. The population data from the other health data tables was not added as it contained the same values.

The above steps are repeated for the remaining 4 data tables.

In [ ]:
# Allocating health data to columns within the detroit_tracts shapefile
detroit_tracts = detroit_tracts.merge(life_expect[['Geo ID','Population','Life Expectancy']], left_on='GEOID', right_on='Geo ID',how='left')
detroit_tracts['life_expcy'] = detroit_tracts['Life Expectancy']
detroit_tracts['total_pop'] = detroit_tracts['Population']
detroit_tracts = detroit_tracts.drop(columns=['Geo ID','Population','Life Expectancy'])

Once the health datasets have been added to the census tract shapefile, an additional step needs to be taken to convert the values from strings to integers to be able to use math functions on them. This is completed by creating a new function called 'conversion', which will convert the string columns from the health data tables to integer columns. 

In [ ]:
# Defining function 'conversion' to convert health data fields from strings into integers
def conversion(field):
    if pd.isna(field) or field == '':
        return np.nan
    if isinstance(field,str):
        return int(float(field.replace('%','')))
    return int(field)

This function is then applied to the health data columns using the apply() function. The total population and life expectancy columns are only converted with the use of conversion, while the remaining 4 columns take 2 additional steps. Since the 4 health data columns are in percentage format (determining the percentage of people that have been diagnosed with the specified condition), they need to be converted into numerical format. This is done by dividing by 100 to convert them into values below 1, before mulitplying by the total population and decimal percentage to determine the total number of people diagnosed. 

The steps taken for the health conditions can be seen below and are taken 3 more times for COPD, coronary heart disease, and cancer respectively.

In [ ]:
# Applying conversion function to health data
# Health risk data is given a numerical value by multplying the percentage by the population
detroit_tracts['total_pop'] = detroit_tracts['total_pop'].apply(conversion)
detroit_tracts['life_expcy'] = detroit_tracts['life_expcy'].apply(conversion)

detroit_tracts['asthma_pct'] = detroit_tracts['asthma'].apply(conversion) / 100
detroit_tracts['asthma'] = detroit_tracts['total_pop'] * detroit_tracts['asthma_pct']

Once the health data has been integrated, the residential shapefile where the data will be reallocated to can be constructed. The land use shapefile is once again filtered, this time to only include residential lots. These residential lots are then dissolved into 1 multipolygon and overlayed by the census tract shapefile. This splits the 1 multipolygon into numerous multipolygons, each of which is fully contained within 1 census tract.

The area of each multipolygon is then calculated to determine the 'total_area' column, containing the total area dedicated to residential lots within each census tract.

In [ ]:
# Filtering zoning shapefile to only include residential lots
residential = zoning[zoning['ZONING_REV'].str.startswith('R')]
residential = residential.dissolve()

# Overlaying the residential and census tract data with gpd.overlay
residential_census = gpd.overlay(residential,detroit_tracts,how='intersection')

# Calculating total area of residential areas within each census tract
residential_census['total_area'] = residential_census.geometry.area

An additional 2 overlays are then completed to create *residential_inside* and *residential_outside*. The 2 functions are split based on whether the residential area is contained within a buffer, using intersection and difference to prevent overlay between the 2 shapefiles.

To aid with future analysis, a new column called 'buffer' is created within both shapefiles, with shapefiles inside a buffer given an 'I' and shapefiles outside a buffer given an 'O'. The 2 shapefiles are then concatenated into 1 shapefile, merging the columns together and maintaining the geometries of both shapefiles.

In [ ]:
# Creating two overlays based on if residential_area 
residential_inside = gpd.overlay(residential_census,buffer_gdf,how='intersection')
residential_outside = gpd.overlay(residential_census,buffer_gdf,how='difference')

# Setting up 'buffer' column to determine if polygon is within brownfield buffer
residential_inside['buffer'] = 'I'
residential_outside['buffer'] = 'O'

# Using pd.concat to join the two dataframes and their columns together into one dataframe
residential_final = gpd.GeoDataFrame(pd.concat([residential_inside,residential_outside],ignore_index=True),crs=residential_inside.crs)

# Calculating area of the polygons
residential_final['polygon_area'] = residential_final.geometry.area

In addition to calculating the total area of residential lots per census tract, a city-wide analysis on the total area of residential lots and the total area of brownfield buffers is calculated to determine the percentage of residential areas within close proximity to a brownfield site. The total area for both values is calculated using the *residential_final* and *residential_inside* shapefiles. The borders of both are first dissolved under new variable names *residential_total* and *buffer_total* respectively, before the total area is calculated. The total area of both are then divided to determine the total percentage of impacted residential lots, 50.30%.

In [ ]:
# Dissolving each shapefile into 1 multipolygon
residential_total = residential_final.dissolve()
buffer_total = residential_inside.dissolve()

# Calculating the total area of all residential lots and residential lots inside buffer
residential_total['total_area'] = residential_total.geometry.area
buffer_total['total_area'] = buffer_total.geometry.area

# Dividing the inside residential area by total residential area
total_pct = buffer_total['total_area'] / residential_total['total_area']
print(total_pct)

The final major step for this analysis is to conduct areal allocation of the population and health data to fit the new residential shapefile. 3 lines of code are run to set the parameters for areal allocation. The first calculates the area percentage of each polygon by dividing polygon_area and total_area. This determines the percentage of residential land each polygon takes within a census tract. This is used to calculate how much of the total population should be allocated based on the area of the polygon. If a polygon takes 20% of the total area, 20% of the total population will be allocated to the polygon.

The third line goes back to the 'buffer' column created earlier. With the use of lambda within the apply() function, polygons with an 'I' string are given a 60% weighting, while other polygons are given a 40% weighting. This weighting will be applied to the health condition columns, allocating a higher percentage of those diagnosed with a health condition towards residential areas within a buffer.

The life expectancy column was not allocated in this analysis, with the scope remaining at a census tract level.

In [ ]:
# Calculating percentage each residential polygon takes up within the total residential area
residential_final['area_pct'] = residential_final['polygon_area'] / residential_final['total_area']

# Allocating population per polygon based on its area percentage
residential_final['total_pop'] = residential_final['total_pop'] * residential_final['area_pct']

# Adding a higher weighting to polygons within close proximity of a brownfield site
residential_final['risk'] = residential_final.apply(lambda x: 0.60 if x['buffer'] == 'I' else 0.40, axis=1)

Now that the parameters for areal allocation are set, the allocation for the remaining health data can be calculated. First the *asthma* column is multiplied by the *risk* column, containing a value of either 0.6 or 0.4 depending on its location. This calculates how much of the total asthma population should be allocated based on the given weighting. The column value is further trimmed down by multiplying the column again, this time by *area_pct*. The result gives an estimated number of residents diagnosed with asthma within a residential area.

The steps below are repeated for the 3 remaining columns.

In [ ]:
# Using weighting to allocate higher percentage to polygons near brownfield sites
residential_final['asthma'] = residential_final['asthma'] * residential_final['risk']

# Calculating the total number based on area percentage
residential_final['asthma'] = residential_final['asthma'] * residential_final['area_pct']

Now that the analysis is complete, they can be visualized using pyplot. The code below displays the distribution of asthma cases within Detroit. The other maps shown later in the notebook were also generated using pyplot.

In [ ]:
# Plotting results for asthma diagnoses
fig, ax = plt.subplots(figsize=(10,6))

residential_final.plot("asthma", scheme='quantiles', cmap='Reds', ax=ax)

ax.set_title('Distribution of Asthma Diagnoses in Detroit, Michigan', fontsize=14, fontweight='bold')
plt.colorbar(ax.collections[0], ax=ax, label='Asthma Diagnoses', orientation='vertical', ticks=[])
ax.set_axis_off()

plt.show()

## Results and Discussion

A total of 7 maps were created displaying the distribution of brownfield sites, population distribution, and health demographics. This section will analyze the results of the 7 maps, note some common patterns, and note some of the issues and limitations within the project.

### Maps and Discussion of Findings

![alt text](Brownfield_Sites-1.png)

The main finding from this map is that of the over 600 current brownfield sites within Detroit, many are found in large clusters. 2 hotspots of note are along the Detroit River Waterfront south of the downtown district and Northern Detroit near the border between Wayne and Macomb Counties. These hotspots show that many brownfield sites are found within close proximity of each other, which is to be expected as the surrounding area is most likely industrial zoning as well. And while this means that not as many residential areas are exposed to brownfield sites as they're could've been if industrial districts were more spread out, it may also prove challenging to clean up these industrial areas if plans to convert these districts into residential or mixed-use development materializes. 

![alt text](Population_Distribution-2.png)

The second map of population distribution further reinforces that density of brownfield sites and population are often negatively correlated. There are some notable exceptions however, specifically around the southern tracts of Detroit where the total population is higher than the city average yet there is a high density of brownfield sites. This can be tied to Detroit's historical importance as a hub for shipping manufactured goods along the Great Lakes. Hence building industrial development along the Detroit River was to be expected and explains the high number of brownfield sites within the area.

![alt text](Life_Expectancy-1.png)

The Life Expectancy map contained some noticable trends, the most glaring being that life expectancy is higher on average when closer to the downtown core. The opposite was noticed in the outskirts of the city as life expectancies were on the lower side. The former can be partially explained as the neighborhoods surrounding downtown Detroit are experiencing gentrification and contain wealthier residents. The density of brownfield sites doesn't seem to impact the life expectancy much, as some of the highest life expectancies were witnessed in census tracts south of the downtown core, which as mentioned earlier contain a high number of brownfield sites. However findings in West Detroit seem to support the notion that brownfield sites can lead to lower life expectancies.

![alt text](Asthma_Distribution-2.png)

Upon initial glance, the findings from the asthma map indicate that there is a positive correlation between brownfield sites and higher asthma rates. Many of the noted industrial clusters within Detroit contain higher asthma rates when compared to census tracts with a lower number of brownfield sites. The industrial clusters within West Detroit, along the Detroit River, and the northern outskirts all display a higher number of asthma rates compared to their neighboring areas. One interesting find is in Northeast Detroit near the county border, where asthma rates are higher outside of the buffer zones than in inside. This can likely be attributed to the areal allocation method used that allocated data based on the area.

![alt text](COPD_Distribution-1.png)

The COPD map displays similar results to the asthma map, albeit with some slight variance between some of the census tracts. The most notable differences between the two maps is that the COPD rates in Northeast Detroit are slightly lower in the outer buffer areas, but the numbers are still higher than compared to areas within the buffer zone. The other noticable trends are that an industrial cluster slightly west does contain higher COPD rates than asthma rates, in addition some census tracts near the Dearborn reported similar findings. However, the general trend does seem to indicate that city-wide COPD rates are lower than asthma rates, albeit marginally so. 

![alt text](CHD_Distribution-1.png)

The CHD map shows the same health demographic trends as the previous 2 maps, albeit to some slight variance. The two most notable are the census tracts near the Dearborn border and a census tract just east of a small linear cluster in Northwest Detroit. The Dearborn census tracts registered lower CHD rates when compared to Asthma and COPD rates, while the census tract in Northwest Detroit registered higher CHD rates than asthma or COPD rates. This is an interesting find particulary since the asthma rates are lower than CHD rates. Perhaps the browmfield sites just west of the census tract might've played a role, especially since they seem to be concentrated along an arterial in the area.

![alt text](Cancer_Distribution-1.png)

The cancer distribution map shows the lowest rates of all 4 health diagnoses maps, as even many of the census tracts along industrial clusters saw lower cancer rates than the other 3 health risks. However the trend does remain the same as tracts surrounding brownfield sites experienced higher cancer rates. Two areas however saw higher cancer rates than many of the other health risk maps: the Dearborn border tracts and additional tracts east of the linear industrial cluster. The cancer rates within the Dearborn border tracts were consistently higher, in contrast to asthma and COPD rates where one location was a hotspot and the others scored lower. At the linear cluster, additional census tracts east of the cluster reported high cancer rates, further implying that perhaps the linear cluster of brownfield sites might be impacting the health of the residents in these eastern tracts.

## Issues and Limitations

Even though the findings from this analysis indicate that there is a correlation between brownfield sites and increased health risks, there are some notable issues and limitations present within the project.

### Issues

One of the biggest issues encountered was that the health data acquired from the PHOENIX Project was structured in percentages, but also in string format. While converting the data into numerical value and into integer columns was doable, it proved troublesome initially. If I had to make a suggestion for improving the way data is stored in the PHOENIX Project, offering the data in either numerical or integer format would've made the analysis process less tedious.

Another issue that was present was that the areal allocation used in the analysis sometimes allocated a larger percentage of health data to areas outside of the brownfield buffer zone. This issue can largely be stemmed from the areal allocation method relying too heavily on the area of a polygon. Perhaps the weighting can also attributed to this issue as the weighting for polygons within the buffer area could've been given a higher weighting. Other parameters could've also been used to further prevent the possibility of polygons outside of the buffer zone containing higher health risk than areas within the buffer zones.

### Limitations

In addition to issues, some limitations were also present.

One of the biggest limitations present was the zoning shapefile. Even though the shapefile was useful in determining the location of residential areas within Detroit, it didn't visualize vacancy rates within the city. Due to this, results within census tracts with high vacancy rates could very likely display different results if vacant lots were excluded from the analysis.

Another limitation to the project is the use of census tracts, as the use of blocks or block groups could've offered more detailed results on a block-by-block basis and proved more effective in analyzing the long-term health impacts of brownfield sites. However, this level of analysis is currently limited (at least on an open data level) as census tract data is currently the smallest kinds of demographic boundaries you can use for health analysis within Detroit. Perhaps in the future the PHOENIX Project will collect health data at a block group or block level, which would allow for a more thorough and detailed version of this analysis to become more feasible.

A third limitation was in terms of some of the data acquired from the PHOENIX Project. Many census tracts, for example, contained no life expectancy data despite large population numbers being recorded in select census tracts. A limitation regarding the use of recent PHOENIX Project data with recent census tracts shapefile was also present as many census tracts carried over no data when a 2024 census tract shapefile was used. However when a 2019 census tract shapefile was used, few census tracts contained no values. I'm inclined to believe that this might've been an oversight on my part, but it is possible that despite new data, this data is being added to an outdated census tract shapefile that has since been replaced by a more recent version.

## Conclusion

This analysis determined that the impact of brownfield sites left over from the departure of manufacturing jobs is noticeable on a demographic scale as many of the most vulnerable tracts were located within close proximity of brownfield clusters. Despite this, some health data including life expectancy didn't seem to have as significant of an impact as initially thought, especially on the census tracts in South Detroit. 

Even though the analysis can conclude that brownfield sites increase the risk of health conditions including asthma, cancer, COPD, and CHD, future studies on the same topic can expand more and conduct more detailed and thorough research, especially if health data on a block group level becomes more readily available. Future studies like these could also utilize other programs including ArcGIS Pro or other specialized Python packages for geospatial analysis like ArcPy were used to conduct the study.

A web map of the analysis' findings can be found here:
*https://aterech.github.io/brownfield_sites_detroit*

## Sources
City of Detroit Open Data Portal (2023). *City of Detroit Boundary*. City of Detroit

City of Detroit Open Data Portal (2025). *Zoning Official*. City of Detroit

Data Driven Detroit (2014). *Brownfields*.

Eicher, C.L. & Brewer, C.A. (2001) *Dasymetric Mapping and Areal Interpolation: Implementation and Evaluation, Cartography and Geographic Information Science*,
28:2, 125-138, DOI: 10.1559/152304001782173727

Lodge, E.K. Guseh, N.S. Martin, C.L. Fry, R.C. White, A.J. Ward-Caviness, C.K. Galea, S. Aiello, A.E. (2022). *The effect of residential proximity to brownfields, highways, and heavy traffic on serum metal levels in the Detroit Neighborhood Health Study*. Environmental Advances, 9(100278), pp. 1-11.
https://doi.org/10.1016/j.envadv.2022.100278 

US Census Bureau (2019). *TIGER Shapefile, Census Tract (Michigan)*. US Census Bureau.

Wayne State University (2019-2024). *PHOENIX Project*. Darkhorse Analytics.